# SESION 4 DICIEMBRE

# Modificamos columnas que eran object a float 

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_rawdata = pd.read_csv("raw_data_20231201.csv", index_col=0)

In [4]:
def cambiar_comas(cadena):
    try:
        return float(cadena.replace(",", "."))
    except:
        return np.nan
    
df_rawdata["Worklifebalance"] = df_rawdata["Worklifebalance"].apply(cambiar_comas)
df_rawdata["Monthlyincome"] = df_rawdata["Monthlyincome"].apply(cambiar_comas)
df_rawdata["Performancerating"] = df_rawdata["Performancerating"].apply(cambiar_comas)
df_rawdata["Totalworkingyears"] = df_rawdata["Totalworkingyears"].apply(cambiar_comas)

In [5]:
df_rawdata["Employeenumber"]= df_rawdata["Employeenumber"].apply(cambiar_comas)

In [6]:
df_rawdata["Employeenumber"].isnull().sum()

431

# Gestión de nulos

Cambiamos los nulos de Employeenumber con un "auto-incremental"

In [7]:
# Encuentra el último ID existente
ultimo_id = df_rawdata['Employeenumber'].max()
# Define una variable para el siguiente ID después del último
siguiente_id = ultimo_id + 1
# Itera sobre los índices del DataFrame
for indice in df_rawdata.index:
    if pd.isnull(df_rawdata.at[indice, 'Employeenumber']):  # Verifica si el valor es nulo
        df_rawdata.at[indice, 'Employeenumber'] = siguiente_id  # Asigna el siguiente ID
        siguiente_id += 1  # Incrementa el siguiente ID
df_rawdata["Employeenumber"] = df_rawdata["Employeenumber"].astype(int)
# Muestra el DataFrame actualizado
print(df_rawdata["Employeenumber"].tail(10))

1604     927
1605     941
1606     956
1607    2497
1608     966
1609     967
1610     972
1611    2498
1612     990
1613    2499
Name: Employeenumber, dtype: int32


In [8]:
# Muestra el DataFrame actualizado
df_rawdata["Employeenumber"].dtype

dtype('int32')

In [9]:
nulls_cat = df_rawdata[df_rawdata.columns[df_rawdata.isnull().any()]].select_dtypes(include = "O").columns
nulls_cat

Index(['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus',
       'Overtime'],
      dtype='object')

In [10]:
(df_rawdata[['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus','Overtime']].isnull().sum() / df_rawdata.shape[0]) * 100

Businesstravel    47.880795
Department        81.258278
Educationfield    46.490066
Maritalstatus     40.264901
Overtime          41.721854
dtype: float64

In [11]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulls_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_rawdata[col].value_counts() / df_rawdata.shape[0])
    print("........................")

La distribución de las categorías para la columna BUSINESSTRAVEL


Businesstravel
travel_rarely        0.364901
travel_frequently    0.101325
non-travel           0.054967
Name: count, dtype: float64

........................
La distribución de las categorías para la columna DEPARTMENT


Department
 Research & Development     0.122517
 Sales                      0.055629
 Human Resources            0.009272
Name: count, dtype: float64

........................
La distribución de las categorías para la columna EDUCATIONFIELD


...

........................
La distribución de las categorías para la columna MARITALSTATUS


Maritalstatus
Married     0.270861
Single      0.202649
Divorced    0.123841
Name: count, dtype: float64

........................
La distribución de las categorías para la columna OVERTIME


Overtime
No     0.422517
Yes    0.160265
Name: count, dtype: float64

........................


In [12]:
nulls_cat = ['Businesstravel', 'Educationfield', 'Maritalstatus', 'Overtime']

Todos desconocidos!

In [13]:
for columna in nulls_cat:
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df_rawdata[columna] = df_rawdata[columna].fillna("Unknown")

In [14]:
df_rawdata["Overtime"].unique()

array(['No', 'Unknown', 'Yes'], dtype=object)

# Columna Department: vinculada con Jobrole

In [15]:
# primero vemos cómo está la columna Jobrole
df_rawdata["Jobrole"].head()

0     resEArch DIREcToR 
1               ManAGeR 
2               ManaGER 
3     ReseArCH DIrECtOr 
4       sAleS EXECUtIve 
Name: Jobrole, dtype: object

In [16]:
# capitalize valores de Jobrole (primera letra de cada palabra)
df_rawdata['Jobrole'] = df_rawdata['Jobrole'].apply(lambda x: ' '.join(word.capitalize() for word in x.lower().split()))
df_rawdata["Jobrole"].head()

0    Research Director
1              Manager
2              Manager
3    Research Director
4      Sales Executive
Name: Jobrole, dtype: object

In [ ]:
# hacemos función para asignar los valores nulos en Department según la columna Jobrole
def fill_department(df, jobrole, department):
    df.loc[df['Jobrole'] == jobrole, 'Department'] = df.loc[df['Jobrole'] == jobrole, 'Department'].fillna(department)

# Uso de la función fill_department para reemplazar los valores faltantes en 'Department'
fill_department(df_rawdata, 'Research Director', 'Research & Development')
fill_department(df_rawdata, 'Healthcare Representative', 'Research & Development')
fill_department(df_rawdata, 'Laboratory Technician', 'Research & Development')
fill_department(df_rawdata, 'Manufacturing Director', 'Research & Development')
fill_department(df_rawdata, 'Research Scientist', 'Research & Development')
fill_department(df_rawdata, 'Sales Executive', 'Sales')
fill_department(df_rawdata, 'Sales Representative', 'Sales')
fill_department(df_rawdata, 'Human Resources', 'Human Resources')
fill_department(df_rawdata, 'Manager', 'Unknown')

# Mostrar el DataFrame actualizado
df_rawdata[["Jobrole", "Department"]]

In [18]:
# Mostrar estadísticas descriptivas y conteo de valores únicos
print("Estadísticas descriptivas para 'Jobrole':\n", df_rawdata['Jobrole'].describe())
print("\nEstadísticas descriptivas para 'Department':\n", df_rawdata['Department'].describe())

print("\nConteo de valores únicos para 'Jobrole':\n", df_rawdata['Jobrole'].value_counts())
print("\nConteo de valores únicos para 'Department':\n", df_rawdata['Department'].value_counts())

Estadísticas descriptivas para 'Jobrole':
 count                1510
unique                  9
top       Sales Executive
freq                  336
Name: Jobrole, dtype: object

Estadísticas descriptivas para 'Department':
 count                       1510
unique                         7
top       Research & Development
freq                         760
Name: Department, dtype: object

Conteo de valores únicos para 'Jobrole':
 Jobrole
Sales Executive              336
Research Scientist           300
Laboratory Technician        264
Manufacturing Director       148
Healthcare Representative    137
Manager                      105
Sales Representative          84
Research Director             83
Human Resources               53
Name: count, dtype: int64

Conteo de valores únicos para 'Department':
 Department
Research & Development      760
Sales                       341
 Research & Development     185
Unknown                      86
 Sales                       84
Human Resources       

# Todas las columnas categóricas listas 